In [ ]:
# 1. Gerekli Kütüphaneler
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pickle

# 2. Verileri Yükleme
patients = pd.read_csv("patients.csv")
doctors = pd.read_csv("doctors.csv")
appointments = pd.read_csv("appointments.csv")

# 3. Kullanıcı-Doktor Etkileşim Matrisi
# Sadece Completed (tamamlanmış) randevular
completed_appointments = appointments[appointments["Status"] == "Completed"]

# Hasta-Doktor matrisi
user_doctor_matrix = completed_appointments.pivot_table(
    index='Patient_ID',
    columns='Doctor_ID',
    values='Doctor_Choice_Num',
    aggfunc='mean'  # Eğer bir hasta bir doktora birden fazla gitmişse ortalamasını alıyoruz
)

user_doctor_matrix = user_doctor_matrix.fillna(0)  # Boşlukları 0 ile doldur


# 4. Hasta-Hasta Benzerlik Matrisi (Cosine Similarity)
user_similarity = cosine_similarity(user_doctor_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_doctor_matrix.index, columns=user_doctor_matrix.index)

print("✅ Model başarıyla oluşturuldu.")

# 5. Fonksiyon: Doktor Öner
def recommend_doctor(patient_id, user_doctor_matrix, user_similarity_df, doctors_df, top_n=3, specialty_filter=None):
    if patient_id not in user_doctor_matrix.index:
        return "Hasta bulunamadı."

    # Benzer hastaları bul
    similar_patients = user_similarity_df[patient_id].sort_values(ascending=False)[1:]

    # Benzer hastaların sevdiği doktorları bul
    recommended_doctors = pd.Series(dtype=float)

    for other_patient, similarity in similar_patients.items():
        patient_ratings = user_doctor_matrix.loc[other_patient]
        patient_ratings = patient_ratings[patient_ratings == 1]
        recommended_doctors = pd.concat([recommended_doctors, patient_ratings * similarity])

    # Şu anda tercih edilmeyen doktorlar arasından seç
    existing_doctors = user_doctor_matrix.loc[patient_id][user_doctor_matrix.loc[patient_id] == 1].index.tolist()
    recommended_doctors = recommended_doctors.drop(labels=existing_doctors, errors='ignore')

    # Skora göre sırala
    recommended_doctors = recommended_doctors.groupby(recommended_doctors.index).sum()
    recommended_doctors = recommended_doctors.sort_values(ascending=False)

    # Doktor bilgilerini ekleyelim
    recommended_doctors_df = pd.DataFrame({
        'Doctor_ID': recommended_doctors.index,
        'Score': recommended_doctors.values
    })

    recommended_doctors_df = recommended_doctors_df.merge(doctors_df, on='Doctor_ID')

    # Eğer bir branş filtresi verilmişse, ona göre filtrele
    if specialty_filter:
        recommended_doctors_df = recommended_doctors_df[recommended_doctors_df['Specialty'] == specialty_filter]

    return recommended_doctors_df[['Doctor_ID', 'Doctor_Name', 'Specialty', 'Hospital', 'Score']].head(top_n)


patient_id = int(input("Hasta ID'sini girin: "))
specialty_input = input("İstediğiniz branşı yazın (Dahiliye, Jinekoloji, Pediatri, Göz Hastalıkları, Kulak Burun Boğaz, Cildiye, Kardiyoloji, Ortopodi, Nöroloji, Psikiyatri, Üroloji, Genel Cerrahi ")

recommendations = recommend_doctor(
    patient_id,
    user_doctor_matrix,
    user_similarity_df,
    doctors,
    top_n=5,
    specialty_filter=specialty_input
)

print(recommendations)

# 6. Modeli Kaydetme (isteğe bağlı)
with open('user_similarity_model.pkl', 'wb') as f:
    pickle.dump(user_similarity_df, f)

with open('user_doctor_matrix.pkl', 'wb') as f:
    pickle.dump(user_doctor_matrix, f)



✅ Model başarıyla oluşturuldu.
     Doctor_ID     Doctor_Name Specialty  \
24         482    Mürit Yüksel  Nöroloji   
33         471  Emirşan Tarhan  Nöroloji   
68         304   Simten Hançer  Nöroloji   
89          19     Muhip Güçlü  Nöroloji   
101        314     Kaver Akçay  Nöroloji   

                                              Hospital     Score  
24   Dışkapı Yıldırım Beyazıt Eğitim ve Araştırma H...  0.550347  
33                Haseki Eğitim ve Araştırma Hastanesi  0.512316  
68   Bakırköy Dr. Sadi Konuk Eğitim ve Araştırma Ha...  0.425012  
89                      Bursa Çekirge Devlet Hastanesi  0.372521  
101                     Bursa Çekirge Devlet Hastanesi  0.323334  
